## PythonのGIL
+ Pythonは完全な並行処理言語(C言語ライブラリとの親和性とセキュリティを考慮した結果). ちなみに, Rubyも完全並行処理言語.
+ ある時刻にOSにインタープリターが依頼してスレッドリソースを貰って処理をする.
+ GILはOSへスレッドリソースを依頼するための権利.
+ GILは実行中のPythonプログラムで, GIL回収ルールを確認して条件を満たしていた時にコンテキストスイッチを行ってOSから貰った別スレッドで動作する.
+ 常にGILによってスレッドリソースを切り替えながら処理実行をしている.

### GIL回収ルール
1. I/O操作が発生
2. 一定時刻を経過(デフォルト5ms)
+ グローバル変数`gil_drop_request`が設置され, `0`の時は実行し続けて大丈夫だが, `1`になったらGILを回収.
+ `cv_wait(gil, TIMEOUT)`で`gil_drop_request`を制御. `TIMEOUT`でGIL回収条件の経過時間を変更できる.

無限ループによる完全並行処理言語であることを確認

In [ ]:
import threading
import multiprocessing


def loop():
    x = 0
    while True:
        x = x ^ 1


for i in range(multiprocessing.cpu_count()):
    t = threading.Thread(target=loop)
    t.start()

### I/OバウンドなPythonプログラムとCPUバウンドなPythonプログラムによる実行速度効率の違い
| バウンドの種類 | 効率 | 内容 |
| :-- | :-- | :-- |
| CPUバウントなプログラム | × | 元々遅いCPUタスクに加え一定時刻毎のGILによるコンテキストスイッチのオーバーヘッドにより実行効率が悪い |
| I/Oバウンドなプログラム | ○ | I/O処理フェーズに移行した直後にGILによって現スレッドから別スレッドへのコンテキストスイッチが発生し, 実行プログラムをI/O処理と並列に行えるため実行効率が良い |
| CPU・I/Oバウンド混在プログラム | Case by caseだが, Convoy Effectに注意. |

### Convoy Effect
+ 遅いタスクに引きずられて、システム全体が遅くなるという性質.
遅いCPUバウンドタスクと速いIOバウンドタスクをそれぞれ別のスレッドで実行させる時, 処理が速くて本来スレッドの切り替えが不要なIOタスクが, GIL回収ルールによってタスクごと終了前にGILによって実行件を奪われてしまい, 遅いCPUバウンドタスクが終わるまでの間に待ち時間が発生する. I/Oバウンドタスクは既に処理が終了しているのにも関わらず, 次にGILを取得するまで待ってしまう. 結果的に, 遅いCPUバウンドタスクに引きずられてシステム全体が遅くなってしまう.

## PythonではマルチCPUを最大限活用したいならmultithreadingモジュールよりmultiprocessingモジュールを使ったほうが良い